# Mini-projet : Le trésor du donjon
Le but de ce TP est de réaliser un jeu graphique à l'aide de la bibliothèque `pygame`.
Le travail doit être réalisé par groupe de 2. Chaque groupe doit rendre un seul notebook qui présente :
- La règle du jeu.
- La démarche de projet et les choix de programmation :

    * Quelles sont les classes, méthodes et fonctions qui ont été créées et quelle est leur utilité dans le contete de programmation du jeu.
    * Comment le travail a-t-il été réparti entre les membres du groupe

- Le code python constituant le programme du jeu.

## La règle du jeu
Voici une proposition de règle simple.

__Un trésor est "caché" dans un donjon labyrinthique (de type _arbre binaire_). Un aventurier se matérialise dans ce donjon et doit trouver le trésor.__

Cette règle peut tout à fait être modifiée mais le type de labyrinthe (arbre binaire) est imposé.

## Pour commencer le programme
Deux classes adaptées de ce qui a été vu en cours sont données et permettent de générer une grille de labyrinthe (avec la méthode dite "de l'arbre binaire"). Noter que dans cette version, ___x___ désigne l'index de __colonne__ et ___y___ désigne l'index de __ligne__. Ceci permet de faciliter la manipulation de ces grilles avec pygame.

Vous pouvez modifier ces scripts à votre guise.

Il est conseillé de commencer le travail par la recherche d'un affichage du donjon sous forme graphique avec pygame.

## Régle(s) et contenus du jeu :

Le jeu se constitue de __3 éléments majeurs__ :

    -Un menu au démarrage du jeu permettant de sélectionner le mode de jeu (pour le moment le seul mode de jeu jouable est dénommé 'EndLess'
    
    -Un écran pour les deux modes de jeux non dévellopés (perspective d'évolution à préciser)
    
    -Une scène de jeu dans laquelle le joueur peut jouer au jeu en mode "sans fin".
    
La règle du jeu du seul mode jouable ("Endless" ou sans fin en Français) est très simple : Le joueur se déplace dans le __labyrinthe__ et doit y trouver un __diamant__. Dès lors que le joueur se trouve __SUR__ le diamant, le joueur gagne et une partie se relance, un nouveau labyrinte est généré et le trésor est repositionné (aléatoirement dans une zonne carrée en bas a droite du labyrinthe).

Les contrôles tant qu'à eux sont élémentaires : 

    -Flèches directionnelles du clavier pour déplacer le joueur
    
    -La touche échap pour revenir au menu (/!\ Attention, quitter la partie entraine la rénitialisation du labyrinthe, 
    dès lors le joueur est repositionné dans le coin supérieur gauche, la composition du labyrinthe change, et le trésor est replacé aléatoirement).

## Première cellule : algorithme de génération revue

Lors de la conception du jeu, dans un premier temps nous avous analysés le programme qui était à notre disposition à savoir les classes NoeudCellule() ainsi que Grille(). Nous avons observés que l'algorithme de génération aléatoire avait deux défauts majeurs, à savoir:
 
    -Deux lignes sans murs sur les bords haut et gauche du labyrinthe
 
    -Un schéma en "escalier" dans la composition du labyrinthe, visible sur les plus grands d'entre eux mais toujours   
    présente dans les formes plus petites.
    
Pour remédier à cela, Wiktor à pensé à un algorithme récursif qui génère le labyrinthe et la composition des murs de chaque cellules, toujours de type
___ _arbre binaire_ ___ . Pour cela il utilise les deux fonctions Recurs() et RecursXY() dans la méthode de class _init_ de l'objet grille(). Puisque Corentin avait déjà développé l'affichage du labyrinthe depuis l'algorithme d'origine, Wiktor n'a pas eu besoin d'adapter la méthode _str_ pour constater si la génération donnait le résultat attendue ou non. C'est donc pour cela que nous avons décidé de la retiré tout simplement, puisque pour tester la génération nous passions  directement par Pygame.

Nous avons également ajoutés les attributs suivant:

    -Dans la class NoeudCellule():
    
        *les attributs gauche et droite pour la construction en arbre binaire. Correspondant respectivement aux noeuds 
        fils droit et gauche.
        
        *L'attribut pos qui est un tuple de la position x et y de la cellule.
        
        *L'attribut mur sous forme de dictionnaire avec deux clés : N pour les murs nord et O pour les murs ouest
         à la seule différence au code fournit en cours, les True et False sont remplacés par 0 et 1 afin de faciliter la 
         compréhension
         
Dans la class Grille(), une nouvelle méthode y a été implémenté : GetMurs() qui sert à savoir si la cellule désigné à des murs Nord et/ou Ouest.

In [10]:
########
import pygame
from pygame.locals import *
import random
class NoeudCellule() :
    def __init__(self, pos, murs):
        self.pos=pos
        self.gauche=None
        self.droit=None
        self.murs=murs
        
    def __repr__(self):
        """permet d'afficher un arbre sous forme d'une liste"""
        if self==None:
            return None
        else :
            return str([self.pos, self.gauche, self.droit, self.murs])
        
    def creerfd(self, pos, murs):
        '''crée et retourne un noeud fils droit ayant pour valeur le paramètre cle'''
        assert self.droit is None,"Le Noeud possède déjà un fils droit"
        self.droit=NoeudCellule(pos, murs)
        return self.droit
        
    def creerfg(self, pos, murs):
        '''crée et retourne un noeud fils gauche ayant pour valeur le paramètre cle'''
        assert self.gauche is None,"Le Noeud possède déjà un fils gauche"
        self.gauche=NoeudCellule(pos, murs)
        return self.gauche

In [11]:
########      
class Grille :
    """
    Classe permettant de générer un labyrinthe avec la méthode "arbre binaire"
    """
    def __init__(self, nx, ny):
        """
        construction d'une grille labyrinthique de dimension (nx - largeur, ny - hauteur)
        """
        self.nx = nx
        self.ny = ny
        CoorList=[]
        for x in range(nx): 
            for y in range(ny):
                CoorList.append((x, y))
        item=NoeudCellule(CoorList[0], {'N' : True, 'O' : True})
        Recurs(item, CoorList)
        self.grille = item
    
    def GetMurs(self, x, y, arbre):
        
        coordo=(x, y)
        
        def parcours(coord, arbre):
            if arbre.pos==coord:
                global resu
                resu=arbre.murs
            if arbre.gauche!=None:
                parcours(coord, arbre.gauche)
            if arbre.droit!=None:
                parcours(coord, arbre.droit)
        parcours(coordo, arbre)
        return resu

Etant donné la compléxité de l'algorithme récursif, il est important de se pencher dessus afin de bien comprendre, à commencer par la logique même de la construction du labyrinthe <br> La génération commence toujours à 0,0 par soucis de facilité et de gestion, mais le point de début n'influe pas sur le reste de la génération. L'algorithme doit prendre une première décision : avancer d'abord sur l'axe des abcisses, ou celui des ordonnées ? Ce choix est fait aléatoirement, soit x puis y soit y puis x. Une fois un premier axe sélectionné, l'algorithme choisit d'avancer en "avant" ou en "arrière" (+1 ou -1 sur l'axe donné), choix fait encore une fois aléatoirement.<br>
Si aucune erreur n'est rencontrée, la cellule/noeud correspondant devrait être créée, fils gauche pour les abcisses, fils droit pour les ordonnées, cependant, si l'algorithme décide d'avancer ___ET___ reculer sur le même axe (donnant une forme de T), il ne peut pas les stocker au même endroit, ce qui nous pousse vite à abandonner l'idée de x=gauche; y=droit, les seules valeurs à prendre en compte sont l'ordre des cellules couplées à leur position dans la grille, permettant une création de labyrinthe non-linéaire.<br>
Autre cas de figure probable, si l'algorithme décide d'avancer de +1, la section dédiée au recul sera ignorée, cependant si avancer est impossible, l'algorithme essaiera ensuite de reculer même si ce n'est pas l'option choisie.<br><br>
L'algorithme prend donc en compte :<ol><li>Sur quel axe avancer en premier (x ou y)</li><li>Reculer ou avancer (+1 ou -1 sur l'axe)</li><li>Si reculer n'a pas été choisit, mais avancer est impossible, reculer</li><li>Si toutes ces opérations éffectuées sur le fils gauche sont impossibles, effectuer sur le fils droit</li><li>Si rien de tout cela n'est possible, la cellule est donc une feuille de l'arbre, ce qui la pousse à remonter récursivement et finir les opérations sur une cellule précédente</li></ol>

Créer un algorithme "aléatoire" prenant en compte tout ces paramètres en un nombre de lignes limité rélève d'un grand défi auquel on a su répondre. Le code est un algorithme "doublement récursif", la fonction Recurs fait appel à RecursXY qui fait appel à la première etc. Ce choix a été fait afin de limiter les lignes mais aussi limiter au maximum les véifications du type "if", permettant une création du labyrinthe plus rapide, pouvant aller jusqu'à 3 secondes pour les plus grands (5 pour la version rudimentaire non optimisée).<br>La cellule 0,0 est d'abord instanciée et lancée dans la boucle Recurs, représentant la racine de l'arbre. Le choix 1 énoncé plus haut est effectué dans Recurs avec la variable "chx" générée grâce à un random. Une fois le choix effectué, le noeud est lancé dans le RecursXY respectif. Les 3èmes et 4èmes paramètres de cette fonction servent à déterminer l'axe emprunté 1,0 pour x et 0,1 pour y.

In [12]:
'''algorithme récursif, construit un arbre aléatoirement à partir d'un noeud et dictionnaire de coordonnées
   chaque noeud possède un attribut position, et murs, qui varie selon le chemin emprunté'''
def Recurs(noeud, dic):
    posx, posy=noeud.pos[0], noeud.pos[1]
    #si le noeud n'a pas été parcouru, le supprime du dictionnaire
    if noeud.pos in dic:
        del dic[dic.index(noeud.pos)]
    chx=random.choice([1, 2])
    #si le choix=1, créer une cellule sur l'axe x, puis l'axe y
    if chx==1:
        RecursXY(noeud, dic, 1, 0)
        #même procédé mais pour l'axe des ordonnées
        RecursXY(noeud, dic, 0, 1)
    #si le choix=2, créer une cellule sur l'axe y, puis l'axe x, même procédé que si chx==1
    else :
        RecursXY(noeud, dic, 0, 1)
        RecursXY(noeud, dic, 1, 0)

Le coeur de l'algorithme est cette fonction permettant toutes les vérifications nécessaires afin de créer un labyrinthe dit "parfait". L'étape 2 est représentée dans la ligne 3. La création commence par le fils gauche, cependant cela n'a aucun impact sur la génération, comme énoncé plus haut, le fils droit est utilisé si celui de gauche est occupé. En suite l'algorithme vérifie si le déplacement est possible et si il correspond au choix dans le cas d'une première tentative de  recul.<br>
Chaque déplacement dans la grille définit les murs d'une cellule, si on se déplace au nord ou à l'ouest, c'est la cellule actuelle qui est affectée, cependant si on se déplace à l'est ou vers le sud, ce sont les cellules avoisinantes qui le sont. Ces changements sont prit en compte à chaque déplacement, l'état des murs étant difficile à vérifier avec seulement la position des cellules dans la grille et dans l'arbre, nous avons préféré le définir directement lors de leurs création.<br> La fonction étant modulaire et dépendant entièrement des déplacements sélectionnés, cet aspect là a du aussi être prit en compte. Lors de la création des murs d'une cellule, il est plutôt facile de les définir vu qu'on est en train de les façonner, mais quand on recule sur les axes, nous affectons les murs déjà présents, mais il est impossible de seulement leur assigner des valeurs puisque celles ci dépendent des déplacements précédents. Nous nous sommes donc servit des multiplications afin de les définir, explication :<br>
Les murs crées sont automatiquement à True, donc à N:1 et O:1, sauf si le déplacement précédent affecte ces cellules, nous nous retrouvons donc avec un des murs à 0, or en mathématiques, le 0 est le seul chiffre qui multiplié par tout autre revient à une somme de 0. Si par exemple nous arrivons par le mur Ouest, nous avons N:1 ; O:0 et si l'algorithme décide de partir vers le nord, les valeurs données à RecursXY sont 0,1 si nous multiplions les deux 0x1=0 et 1x0=0, les deux murs sont donc bien détruits.<br>
Ensuite l'algorithme crée un fils directement dans la fonction Recurs afin de ne pas avoir à stocker de variable, la "boucle" continue ainsi.

In [13]:
def RecursXY(noeud, dic, x, y):
    posx, posy=noeud.pos[0], noeud.pos[1]
    chx2=random.choice([1, 2])
    #si 2ème choix=1, recule d'une cellule sur l'axe des abcisses
    if noeud.gauche==None:
        if (posx-x, posy-y) in dic and chx2==1:
            noeud.murs={'N' : x*noeud.murs['N'], 'O' : y*noeud.murs['O']}
            Recurs(noeud.creerfg((posx-x, posy-y), {'N' : 1, 'O' : 1}), dic)
            #si le fils gauche est occupé, vérifie le fils droit
        elif (posx+x, posy+y) in dic:
            Recurs(noeud.creerfg((posx+x, posy+y), {'N' : x, 'O' : y}), dic)
        #au cas où une incrémentation serait impossible sur le fils gauche, décrémenter
        elif (posx-x, posy-y) in dic:
            noeud.murs={'N' : x*noeud.murs['N'], 'O' : y*noeud.murs['O']}
            Recurs(noeud.creerfg((posx-x, posy-y), {'N' : 1, 'O' : 1}), dic)
    if noeud.droit==None:
        if (posx-x, posy-y) in dic and chx2==1:
            #si le fils droit est libre, y assigne une position
            noeud.murs={'N' : x*noeud.murs['N'], 'O' : y*noeud.murs['O']}
            Recurs(noeud.creerfd((posx-x, posy-y), {'N' : 1, 'O' : 1}), dic)
        elif (posx+x, posy+y) in dic:
            Recurs(noeud.creerfd((posx+x, posy+y), {'N' : x, 'O' : y}), dic)
        elif (posx-x, posy-y) in dic:
            noeud.murs={'N' : x*noeud.murs['N'], 'O' : y*noeud.murs['O']}
            Recurs(noeud.creerfd((posx-x, posy-y), {'N' : 1, 'O' : 1}), dic)

### Cellule Test :

## La class  Personnage

point vocabulaire : 

__sprite__ = Image d'un jeu ,

__sprite sheets__ = Image avec tous les sprites de l'animation d'une ou plusieurs actions

La  class personnage est une class relativement primordial au jeu. En effet elle permet de gérer les données relatives au joueur (position, inventaire, sprites, etc..). Puisque nous disposons d'une class Personnage, il serait envisageable d'avoir deux joueurs jouant en simultané sur le même écran. Nous avons décidé d'utiliser une class pour deux raison : les  données sont très modulables et nous avions besoin de méthodes de class pour gérer les mouvement dans les 4 directions.

Les méthodes gauche(), droite(), haut(), bas() seront appelées respectivement dans le jeu à chaque fois que le programme devra gérer les actions de mouvements, modifiant ainsi les coordonnées et sprite du personnage.

Class rédigé à deux : Corentin s'est occupé de la strucute et Wiktor de l'animation. Cependant nous l'avons pensé ensemble.

In [14]:
class Personnage(pygame.sprite.Sprite):
    """Class personnage permettant de lister les attributs
    d'un personnage tel que sa positon ou son inventaire"""
    def __init__(self):
        super().__init__()
        """Création d'un personnage avec sa position et son inventaire"""
        self.inventaire={"Torche":False} #Objet non présent & non utilisé, perspective d'évolution.
        self.pos=[0,0]
        self.sprite_sheet = pygame.image.load('Sprites/Char/SpriteSheetLink.png').convert_alpha()
        self.spriteslist=[]
        self.mouv=[2, 3, 6, 3, 2, 2, 3, 6, 3, 2] #list du nombre de pixel à décaler dans la direction du
        #mouvement à chaque sprite de l'animation.
        
        #Récupération des sprites du personnage depuis la sprites sheet
        #pour s'en servir dans l'animation dans les méthodes haut, bas, gauche et droite.
        templist=[]
        for i in range(4):
            image = pygame.Surface([96, 104]).convert_alpha()
            image.blit(self.sprite_sheet, (0, 0), (96*i, 0, 96, 104))
            image= pygame.transform.scale(image,(28,28))
            image.set_colorkey((0,0,0))
            templist.append(image)
        self.spriteslist.append(templist)
        for y in range(4):
            templist=[]
            for x in range(10):
                image = pygame.Surface([96, 104]).convert()
                image.blit(self.sprite_sheet, (0, 0), (96*x, (104*y)+104, 96, 104))
                image= pygame.transform.scale(image,(28,28))
                image.set_colorkey((0,0,0))
                templist.append(image)
            self.spriteslist.append(templist)
            
        self.image=self.spriteslist[0][0]
        self.rect = pygame.Rect(4,4,28,28)
        self.JRect=self.rect
    
    #Méthodes pour modifier les coordonnées du personnage et animer son mouvement.
    def gauche(self, group, screen, charect, camera):
        #Mise-à-jour de la position du joueur.
        self.pos[0]-=1
        #boucle pour animer le mouvment
        for i in range(len(self.mouv)):
            self.image=self.spriteslist[2][i]
            self.JRect=self.JRect.move(-self.mouv[i], 0)
            charect=charect.move(-self.mouv[i], 0)
            screen.fill((0,0,0,0))
            self.rect=self.JRect
            group.draw(screen)
            pygame.display.update(self)
            pygame.time.delay(7*i)
            camera=Update(charect, camera)
        screen.fill((0,0,0,0))
        self.image=self.spriteslist[0][1]
        group.draw(screen)
        pygame.display.update(self)
        return camera, charect
        
    def droite(self, group, screen, charect, camera):
        
        self.pos[0]+=1
        #boucle pour animer le mouvment
        for i in range(len(self.mouv)):
            self.image=self.spriteslist[4][i]
            self.JRect=self.JRect.move(self.mouv[i], 0)
            charect=charect.move(self.mouv[i], 0)
            screen.fill((0,0,0,0))
            self.rect=self.JRect
            group.draw(screen)
            pygame.display.update(self)
            pygame.time.delay(7*i)
            camera=Update(charect, camera)
        screen.fill((0,0,0,0))
        self.image=self.spriteslist[0][3]
        group.draw(screen)
        pygame.display.update(self)
        return camera, charect
        
    def haut(self, group, screen, charect, camera):
        
        self.pos[1]-=1
        #boucle pour animer le mouvment
        for i in range(len(self.mouv)):
            self.image=self.spriteslist[3][i]
            self.JRect=self.JRect.move(0, -self.mouv[i])
            charect=charect.move(0, -self.mouv[i])
            screen.fill((0,0,0,0))
            self.rect=self.JRect
            group.draw(screen)
            pygame.display.update(self)
            pygame.time.delay(7*i)
            camera=Update(charect, camera)
        screen.fill((0,0,0,0))
        self.image=self.spriteslist[0][2]
        group.draw(screen)
        pygame.display.update(self)
        return camera, charect
        
    def bas(self, group, screen, charect, camera):
        
        self.pos[1]+=1
        #boucle pour animer le mouvment
        for i in range(len(self.mouv)):
            self.image=self.spriteslist[1][i]
            self.JRect=self.JRect.move(0, self.mouv[i])
            charect=charect.move(0, self.mouv[i])
            screen.fill((0,0,0,0))
            self.rect=self.JRect
            group.draw(screen)
            pygame.display.update(self)
            pygame.time.delay(7*i)
            camera=Update(charect, camera)
        screen.fill((0,0,0,0))
        self.image=self.spriteslist[0][0]
        group.draw(screen)
        pygame.display.update(self)
        return camera, charect
        
		

### Cellule test :

## Autres class et fonctions :

### Class bouton() :

Class relativement simple, composé de trois attribut :  une position (x,y) et d'un sprite. Elle sert à afficher un bouton à une position, avec une image donnée. Elle a une méthode pour afficher le bouton et une méthode permettant de vérifier si la position de la souris ou bien d'un clic est dans la zonne de l'image du bouton. Nous retrouverons ces ojets dans le jeu.

### Class Objet():

Comme pour la class bouton, elle est relativement simple mais très importante dans une perspective d'évolution puisqu'elle nous permettra d'avoir différents objets dans le jeu (interractible ou non) avec leurs méthodes respectives.

### Fonction VerifObj():

Une fonction qui nous permettra de savoir si le joueur se trouve sur un objet. Dans l'avenir nous pensons à rajouter une touche interraction qui interragira avec l'objet en face du joueur, dans quel cas il faudra prendre un compte l'orientation du joueur. Nous n'aurons plus besoin de cette fonction telle qu'elle est.

In [15]:
class bouton():
    """Objet définisant un bouton de l'interface graphique Pygame."""
    def __init__(self,sprite, x, y):
        self.sprite=sprite
        self.x=x
        self.y=y
        
    def draw(self,surface):
        """Afficher le bouton sur la surface souhaité."""
        surface.blit(self.sprite,(self.x,self.y))
        
    def isOver(self,pos):
        """Méthode permettant de savoir si une position est par dessus le bouton.
        Retourne TRUE si elle l'est et FALSE si elle ne l'est pas"""
        if pos[0]> self.x and pos[0] < self.x+self.sprite.get_width():
            if pos[1]>self.y and pos[1]< self.y + self.sprite.get_height():
                return True
        return False
		
class Objet():
    """Class définisant les objets du jeu"""
    def __init__(self, nom, pos):
        self.nom=nom
        self.pos=pos
		
def VerifObj(dico, pos, obj):
    """Fonction permettant de vérifier si le joueur deux coordonnées sont les mêmes,
    elle sera utilisée pour savoir si le joueur est sur la même cellule que l'objet"""
    res=False
    for item in dico:
        if dico.get(obj)==pos:
            res=True
    return res
		

### Cellule test :

## Autres fonctions récurrentes :

### La fonction init_player_tresor () :

Nous avions besoin également en plus de tout ce qui précède, d'une fonction nous permettant de donner une position au trésor puisque lors du développement nous retrouvions ce schéma d'action à plusieurs endroits et cela représente donc un gain de temps et d'espace dans le code considérable.

### La fonction Update () :

Encore une fois, cette fonction représente un gain de temps puisqu'elle sera appellé à chaque déplacement du personnage.

### La fonction DrawDonjon () :

Cette fonction permet d'afficher un labyrinthe depuis un objet grille. Elle se divise en trois temps, représenté par une  fonction local et deux boucles for :

    -L'affichage complet des cellules du labyrinthe avec leur murs Nord ou Ouest si la cellule en a grâce à la fonction récursive drawnint() 
    
    -L'affichage du bord gauche du donjon.
    
    -L'affichage du bord inférieur du donjon.
    
###  La fonction reset () :

Cette fonction sera utilisé lorsque que le joueur gagnera en atteignant le diamant, et remettra toutes les valeurs à leurs états d'origine. Elle nous donne une meilleure vue pour savoir ce que nous réinitialisons à chaque fin de partie.

In [16]:
def init_player_tresor(Tresor,CharacterRect,x,y):
    """Fonction qui permet de définir la position aléatoire du Trésor. Il est placé aléatoirement dans
    le quart inférieur droit de l'écran.
    Retourne un objet trésor de position aléatoire."""
    Tresor=Objet("Trésor", [x, y])
    tx=random.randint(Tresor.pos[0]-(Tresor.pos[0]//4), Tresor.pos[0])
    ty=random.randint(Tresor.pos[1]-(Tresor.pos[1]//4), Tresor.pos[1])
    Tresor.pos[0]=tx-1
    Tresor.pos[1]=ty-1
    Gameboard.fill((0,0,0,1))
    Gameboard.blit(PFront,(CharacterRect.centerx,CharacterRect.centery))
    Labyrinth.blit(TresorImg,((Tresor.pos[0])*32,(Tresor.pos[1])*32))
    return Tresor

def Update(Charac, cam):
    """Fonction permettant de mettre à 'recharger' l'affichage de l'écran."""
    fenetre.fill((0,0,0))
    Game.blit(Labyrinth,(64,64))
    Game.blit(Gameboard,(64,64))
    sub = Game.subsurface(Charac)
    cam=pygame.Surface((160,160),pygame.SRCALPHA)
    cam=Camera.convert_alpha()
    cam.blit(sub,(0,0))
    cam = pygame.transform.scale(cam,(396,396))
    fenetre.blit(cam,(12,12))
    fenetre.blit(Interface,(0,0))
    pygame.display.flip()
    return cam

#Affichage du Donjon
def drawdonjon(Donjon):
    def drawint(Donjon): 
        """Fonction permettant de d'afficher une grille sur la surface Background.
        Prend en paramètres un donjon."""
        coord_x=Donjon.pos[0]*32
        coord_y=Donjon.pos[1]*32
        Labyrinth.blit(SpriteMaze[4],(coord_x,coord_y))#sol
        if Donjon.murs["N"]==True and Donjon.murs["O"]==True:
            Labyrinth.blit(SpriteMaze[0],(coord_x,coord_y))#coin
        elif Donjon.murs["N"]==False and Donjon.murs["O"]==True :
            Labyrinth.blit(SpriteMaze[2],(coord_x,coord_y))#gauche
        elif Donjon.murs["O"]==False and Donjon.murs["N"]==True:
            Labyrinth.blit(SpriteMaze[1],(coord_x,coord_y))#haut
        elif Donjon.murs["O"]==False and Donjon.murs["N"]==False:
            Labyrinth.blit(SpriteMaze[3],(coord_x,coord_y))#vide
        Game.blit(Labyrinth,(64,64))
        if Donjon.gauche!=None:
            drawint(Donjon.gauche)
        if Donjon.droit!=None:
            drawint(Donjon.droit)
    drawint(Donjon.grille)
    

    #Affichage des bords droit du donjon
    coord_x=Donjon.nx*32
    coord_y=Donjon.ny*32
    for y in range(0,coord_y,32):
        Labyrinth.blit(SpriteMaze[2],(coord_x,y))
        Game.blit(Labyrinth,(64,64))
                  
    #Affichage des bords inférieur du donjon
    for x in range(0,coord_x,32):
        Labyrinth.blit(SpriteMaze[1],(x,coord_y))
        Game.blit(Labyrinth,(64,64))

def reset(Joueur,Donjons,Tresor,x,y,CharacterRect, JoueuRect):
    """Fonction permettant de Remettre la position du joueur et de son sprite en (0,0),
    réinitialiser son inventaire, et de replacer aléatoirement le trésor.
    Retourne Tous les objets/variables réinintialisées."""
    Joueur.inventaire["Tresor"]=False
    Joueur.pos[0]=0
    Joueur.pos[1]=0
    Gameboard.fill((0,0,0,1))
    Labyrinth.fill((0,0,0,1))
    CharacterRect.x=0
    CharacterRect.y=0
    JoueuRect.rect=pygame.Rect(4,4,28,28)
    JoueuRect.JRect=JoueuRect.rect
    Tresor=init_player_tresor(Tresor,CharacterRect,x,y)
    Donjons=[]
    Donjons.append(Grille(x,y))
    
    return Joueur,Donjons,Tresor.pos[0],Tresor.pos[1],CharacterRect, JoueuRect

### Cellule test:

## Class  gamestate():

La class gamestate(), une class qui nous a possé beaucoup de problèmes...

En effet, si nous souhaitions avoir ce système de menu/différents modes de jeux en réduisant la complexité de la tache au maximum, il était primordial d'avoir cette class.

A son initiation, nous lui donnons plusieurs attributs qui seront tous réutilisé dans le jeu. Nous y créons le personnage, les sprites, le donjon et la liste de tous les donjons, la taille universel du donjon, la position du Trésor, le dictionnaire des objtes etc..

### La méthode uc :

Elle correspond en anglais à Under Construction, elle est relativement simple puisqu'elle affiche juste une image "En construction" en si on clique n'importe où sur l'écran nous revenons au menu.

### La méthode intro :

Elle est quand à elle un peu plus complex mais reste élémentaire. On y affiche un fond, et les trois boutons. Lorsque la souris passe au-dessus des boutons, leur image change pour indiquer que le curseur est au-dessus. Lorsque l'on clic sur les deux premiers boutons, on est redirigé sur la méthode uc(). Si on clique sur le dernier bouton, on exécute les attribution de variables et les changements d'affichage nécésaire au passage sur l'écran jeu pour pouvoir charger un labyrinthe etc...

### La méthode endless :

   Méthode qui correspond aux actions possibles sur l'écran de jeu. On y affiche dans l'ordre : le sol du labyrinthe, les murs du labyrinth, le trésor, le personnage puis enfin l'interface utilisateur (autour de l'écran du jeu) dans lequels nous comptons y afficher les objets de l'inventaire du joueur, une carte et sûrement d'autres éléments.

   A chaque déplacement, ce qui doit être mis-à-jour l'est. L'intêret d'avoir plusieurs surfaces différentes est que sur une surface nous avons le donjons, sur une autre le personnage (éléments dynamique), et sur une troisième l'interface utilisateur. Ainsi, nous il ne nous reste plus qu'a "blit" ce que nous souhaitons et n'avons pas à reparcourir la grille pour afficher le labyrinthe à chaque mouvement mais à uniquement blit sa surface sur la fenêtre. 
    
   De plus, pour "zoomer" sur le personnage, on utilise une quatrième surface nous permettant grâce à un transform.scale de n'afficher les cases d'un rayon de x (modulable donc). Nous avons opté pour ce choix de caméra afin que le jeu sois jouable sur toutes les résolutions d'écran possible.
    
   Si nous n'avions pas opté pour ce choix-ci, nous aurions été limité dans la taille possible du donjon, grâce à cette solutions, nous sommes uniquement limité par la profondeur de récursivité de l'ordinateur. Nous pouvons donc monter jusqu'à des labyrinthe d'une taille de 25 sur 25 (avec de légers problèmes selon les machines, c'est pourquoi ici nous l'avons laissé à 10). Cependant, il est tout à fait possible de générer des labyrinthes de 7 sur 18 par exemple. Cette solution illustre parfaitement la modularité du jeu recherchée.
   
### La méthode gestionnaire_de_scene () :

Cette méthode est la méthode maîtresse pour ce qui est de passer d'une méthode/scène à l'autre. Elle compare simplement le nom de l'état et apelle la méthode correspondante. Simple mais très éfficace.

In [17]:
class gamestate():
    """Objet du jeu, permettant de passer de scène en scène, c'est-à-dire, par exemple, de passer du menu au jeu."""
    global CharacterRect
    def __init__(self,Camera, CharacterRect):
        #Définitions des variables du jeu        
        self.allstate=["intro","endless","UC"] #Liste de toutes les scènes du jeu
        self.etat = self.allstate[0] #Scène actuel du jeu (index à changer selon la scène souhaité au lancement)
        self.boucle= True
        self.Joueur=Personnage()
        self.my_group = pygame.sprite.Group(self.Joueur)
        self.CharacterRect=CharacterRect
        self.Donjons=[]
        self.x,self.y=10,10
        self.Tresor=None
        self.Tresor=init_player_tresor(self.Tresor,self.CharacterRect,self.x,self.y)
        self.Objets={'Tresor' : self.Tresor.pos}#Dictionnaire des tous les objets présents
        self.Camera=Camera
        self.Camera = pygame.transform.scale(self.Camera,(160,160))
        
        self.Joueur,self.Donjons,self.Tresor.pos[0],self.Tresor.pos[1],self.CharacterRect, self.Joueur=reset(self.Joueur,self.Donjons,self.Tresor,self.x,self.y,self.CharacterRect, self.Joueur)

    def uc(self):
        UC.fill((0,0,0))
        UC.blit(imguc,(300-64,300-52))
        fenetre.blit(UC,(0,0))
        pygame.display.flip()
        for event in pygame.event.get():
            if event.type==QUIT: #Appuie sur la croix.
                self.boucle = False #Sortie de boucle
            if event.type==MOUSEBUTTONDOWN:
                #Sortie de scènes = "Déchargement"
                Labyrinth.fill((0,0,0))
                Gameboard.fill((0,0,0))
                pygame.display.flip()
                self.etat=self.allstate[0]
    
    def intro(self):
        Labyrinth.fill((0,0,0,1))
        Gameboard.fill((0,0,0,1))
        Labyrinth.blit(IntroBG,(0,0))
        btnEndless.draw(Gameboard)
        btnCampagne.draw(Gameboard)
        btnCasual.draw(Gameboard)
        fenetre.blit(Labyrinth,(0,0))
        fenetre.blit(Gameboard,(0,0))
        pygame.display.flip()
        
        for event in pygame.event.get():
            pos=pygame.mouse.get_pos()
            
            if event.type==QUIT:
                self.boucle = False
            if event.type==MOUSEBUTTONDOWN:
                if btnEndless.isOver(pos):
                    #ligne ci-dessous à mettre en commentaire si on souhaite garder le même laby en endless
                    #lorsqu'on retourne au menu
                    self.Joueur,self.Donjons,self.Tresor.pos[0],self.Tresor.pos[1],self.CharacterRect, self.Joueur=reset(self.Joueur,self.Donjons,self.Tresor,self.x,self.y,self.CharacterRect, self.Joueur)
                    Labyrinth.fill((0,0,0,1))
                    Gameboard.fill((0,0,0,1))
                    Game.fill((0,0,0,1))
                    fenetre.fill((0,0,0))
                    drawdonjon(self.Donjons[-1])
                    Gameboard.blit(PFront,self.CharacterRect)
                    Labyrinth.blit(TresorImg,((self.Tresor.pos[0])*32,(self.Tresor.pos[1])*32))
                    Game.blit(Labyrinth,(64,64))
                    Game.blit(Gameboard,(64,64))
                    pygame.display.flip()
                    fenetre.blit(Interface,(0,0))
                    self.etat=self.allstate[1]
                
                if btnCampagne.isOver(pos):
                    self.etat=self.allstate[2]
                if btnCasual.isOver(pos):
                    self.etat=self.allstate[2]
                
            if event.type == pygame.MOUSEMOTION:
                if btnEndless.isOver(pos):
                    btnEndless.sprite=imgBtnEndless[1]
                elif btnCampagne.isOver(pos):
                    btnCampagne.sprite=imgBtnCampagne[1]
                elif btnCasual.isOver(pos):
                    btnCasual.sprite=imgBtnCasual[1]
                else:
                    btnEndless.sprite=imgBtnEndless[0]
                    btnCampagne.sprite=imgBtnCampagne[0]
                    btnCasual.sprite=imgBtnCasual[0]
                    
        
        
    def endless(self):
        """Definition qui correspond à la scène de jeu du mode endless(qui se répète)"""
       
        for event in pygame.event.get():
            if event.type==QUIT:
                self.boucle = False
            if event.type==KEYDOWN:
                
                #Récupération des informations sur les murs aux alentours
                murN=self.Donjons[0].GetMurs(self.Joueur.pos[0],self.Joueur.pos[1], self.Donjons[0].grille)['N']
                murS=self.Donjons[0].GetMurs(self.Joueur.pos[0],self.Joueur.pos[1]+1, self.Donjons[0].grille)['N']
                murO=self.Donjons[0].GetMurs(self.Joueur.pos[0]+1,self.Joueur.pos[1], self.Donjons[0].grille)["O"]
                murE=self.Donjons[0].GetMurs(self.Joueur.pos[0],self.Joueur.pos[1], self.Donjons[0].grille)["O"]
                
                if event.key==K_ESCAPE: #Touche échap
                    Labyrinth.fill((0,0,0))
                    Gameboard.fill((0,0,0))
                    pygame.display.flip()
                    self.etat=self.allstate[0]
                
                if event.key==K_LEFT: #flèche gauche
                    if self.Joueur.pos[0]!=0 and murE==False:
                            Gameboard.fill((0,0,0,0))
                            #Labyrinth.blit(TresorImg,((self.Tresor.pos[0])*32,(self.Tresor.pos[1])*32))
                            self.Camera, self.CharacterRect=self.Joueur.gauche(self.my_group, Gameboard, self.CharacterRect, self.Camera)
                        

                if event.key==K_RIGHT: #flèche droite
                    
                    if self.Joueur.pos[0]+1<self.Donjons[0].nx and murO==False:
                            
                            Gameboard.fill((0,0,0,0))
                            #Labyrinth.blit(TresorImg,((self.Tresor.pos[0])*32,(self.Tresor.pos[1])*32))
                            self.Camera, self.CharacterRect=self.Joueur.droite(self.my_group, Gameboard, self.CharacterRect, self.Camera)

                            
                if event.key==K_UP: #flèche haut
                    if self.Joueur.pos[1]!=0 and murN==False:
                            
                            Gameboard.fill((0,0,0,0))
                            #Labyrinth.blit(TresorImg,((self.Tresor.pos[0])*32,(self.Tresor.pos[1])*32))
                            self.Camera, self.CharacterRect=self.Joueur.haut(self.my_group, Gameboard, self.CharacterRect, self.Camera)

            
                if event.key==K_DOWN: #flèche bas
                    if self.Joueur.pos[1]+1<self.Donjons[0].ny and murS==False:
                            Gameboard.fill((0,0,0,0))
                            #Labyrinth.blit(TresorImg,((self.Tresor.pos[0])*32,(self.Tresor.pos[1])*32))
                            self.Camera, self.CharacterRect=self.Joueur.bas(self.my_group, Gameboard, self.CharacterRect, self.Camera)

                if VerifObj(self.Objets, self.Joueur.pos, "Tresor")==True and self.Joueur.inventaire["Tresor"]==False:
                    self.Joueur.inventaire["Tresor"]=True
                    #retirer le commentaire ci-dessous pour fermer le jeu lors de la récupération du trésor.
                    #pygame.quit()
                    self.Joueur,self.Donjons,self.Tresor.pos[0],self.Tresor.pos[1],self.CharacterRect, self.Joueur=reset(self.Joueur,self.Donjons,self.Tresor,self.x,self.y,self.CharacterRect, self.Joueur)
                    drawdonjon(self.Donjons[-1])
                    Gameboard.fill((0,0,0,1))
                    Gameboard.blit(PFront,self.CharacterRect)
                    Labyrinth.blit(TresorImg,((self.Tresor.pos[0])*32,(self.Tresor.pos[1])*32))
        self.Camera=Update(self.CharacterRect, self.Camera)
        
    def gestionnaire_de_scene(self):
        """Méthode qui permet de basculer d'une scène à l'autre"""
        if self.etat == self.allstate[0]:
            self.intro()
        if self.etat == self.allstate[1]:
            self.endless()
        if self.etat == self.allstate[2]:
            self.uc()

## Initalisation des dernières variables et chargement des images:



In [18]:
pygame.init()

#Variables paramètres
framerate=pygame.time.Clock().tick(60)
pygame.key.set_repeat(500,400)


size = (1500,1500)

#Création de la fenêtre de l'application
fenetre=pygame.display.set_mode((600,600))#fenêtre de taille 640*480
CharacterRect=pygame.Rect(-64,-64,160,160)

#Chargement des murs
SpriteMaze=(pygame.image.load('Sprites/Walls/Corner.png').convert_alpha(),
            pygame.image.load('Sprites/Walls/Upper.png').convert_alpha(),
            pygame.image.load('Sprites/Walls/Left.png').convert_alpha(),
            pygame.image.load('Sprites/Walls/None.png').convert_alpha(),
            pygame.image.load('Sprites/Walls/Floor.png').convert_alpha())

PFront=pygame.image.load('Sprites/Char/PFront.png').convert_alpha()

#Chargement Des sprites Items
TresorImg=pygame.image.load('Sprites/Items/diamant.png').convert_alpha()

#Chargement des interfaces utilisateurs
IntroBG=pygame.image.load('Sprites/Menu/UI/Menu.png').convert_alpha()
imgBtnCampagne=(pygame.image.load('Sprites/Menu/Boutons/Normal/Campagne.png').convert_alpha(),
             pygame.image.load('Sprites/Menu/Boutons/Clicked/Campagne.png').convert_alpha(),)
imgBtnCasual=(pygame.image.load('Sprites/Menu/Boutons/Normal/Casual.png').convert_alpha(),
           pygame.image.load('Sprites/Menu/Boutons/Clicked/Casual.png').convert_alpha())
imgBtnEndless=(pygame.image.load('Sprites/Menu/Boutons/Normal/Endless.png').convert_alpha(),
            pygame.image.load('Sprites/Menu/Boutons/Clicked/Endless.png').convert_alpha())
imguc=pygame.image.load('Sprites/Menu/UI/UC.png').convert_alpha()#image Under Construction
imguc=pygame.transform.scale(imguc,(128,104))

Interface=pygame.image.load('Sprites/Menu/UI/Interface.png').convert_alpha()

#Initialisation des boutons
btnCampagne=bouton(imgBtnCampagne[0],175,265)
btnCasual=bouton(imgBtnCasual[0],175,355)
btnEndless=bouton(imgBtnEndless[0],175,455)


#Définition des surfaces :

#Invisibles
UC=pygame.Surface(size,pygame.SRCALPHA)
UC.blit(imguc,(300-64,300-32))
Labyrinth=pygame.Surface(size,pygame.SRCALPHA) #Background (jamais affichée)
Labyrinth=Labyrinth.convert_alpha()
Labyrinth.fill((0,0,0,0))
Gameboard=pygame.Surface(size,pygame.SRCALPHA) #Surface de tous les éléments interractibles. (jamais affichée)
Gameboard=Gameboard.convert_alpha()

#Affichée
Game=pygame.Surface(size,pygame.SRCALPHA) #Concaténation de Labyrinth et Gameboard
Game=Game.convert_alpha()
Camera=pygame.Surface((160,160),pygame.SRCALPHA) #Surface qui sera affiché dans L'UI
Camera=Camera.convert_alpha()

### Cellule test:

## Lancement du jeu :



In [19]:
game_state = gamestate(Camera, CharacterRect)
pygame.display.flip()
while game_state.boucle:
    game_state.gestionnaire_de_scene()
pygame.quit()

## Cellule test:

## Conclusion et perspectives d'évolution :

Voilà, normalement toutes les cellules sont sensées fonctionner et le jeu doit tourner sans erreurs ! Nous avons encore beaucoup d'ambition quand au développement du jeu accompagné de son lot d'idées. Nous souhaitons axer ce jeu en tant que Rogue-Like (progressions des caractéristiques du personnage mais à chaque mort, réinitialisation de l'avancée sur la carte) mais également en tant que rpg. Pour ce qui est des deux autres modes de jeu, nous souhaitons les axer ainsi :

   -Le mode Casual :
       un  mode de jeu dans lequel le joueur est libre de choisir la génération de son donjon et des éléments de gameplay. Les options de générations seront sauvegardés et pourrons être réutilisées et modifiées par le joueur (nous envisageons de le faire sous forme de fichiers Json). Ce mode de jeu repose sur la modularité du jeu, d'où l'importance plus haut de la recherche de modularité dans le développement du jeu.
       
   -Le mode campagne : 
       Quand à lui, le mode campagne seront plus linéaire. Le joueur suivra une histoire. Le labyrinth sera déjà généré(labyrinthe sur plusieurs étages) avec son lots d'énigmes, d'ennemies (combatables au tour par tour sur une scène prévu à cet effet), et de rebondissement ! La modularité du jeu sera tout autant importante pour le développement du jeu puisque cela nous facilitera les changements possibles en cours de route.
       
Bien évidemment si nous continuons ce projet de notre coté, nous voyons cela également comme un bon moyen de réinvestir les connaisances apprises en cours.

Pour finir, nous préciserons que nous avons travaillés tous les deux dessus. Wiktor s'étant plus attardé sur la partie algorithmique et les animations et Corentin sur le coté affichage, mais lors de la mise en commun régulière de nos bout de codes, chacun travaillait sur l'ensemble, retouchant le code pour l'améliorer et intégrer de nouveaux éléments. Wiktor s'est occupé de la création des sprites sauf pour celui du personnage qui est disponible en libre droits sur internet.